<a href="https://colab.research.google.com/github/nanpolend/365E5/blob/master/%E8%BB%8A%E7%89%8C%E8%BE%A8%E8%AD%98%E6%A8%A1%E7%B5%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>光頭傑夫製作</h1><img src="https://rolqiw.bl.files.1drv.com/y4mYqkObWmQmdJFd8pab8t8m7d_mFhh3_YG63fiAnPaG6gjahHFt0n087pF97x1W2wemm7hd4ZdiPD2OxoPtqcCD_4-Jy-fojnt97kGGWR4Hnq0-9Ny61afd7WVHVTmF_9pcr45OFDFSwLKByeQU1q9TjwKk-HsNmWVsW1eGQ4Gi-9G4pQ5DYoTxswYJ5r5dLF-kKsA1InEyrqmV3kF5pm4WQ?width=658&height=584&cropmode=none" width=160px align="left" />
<!--nicknochnack/ANPRwithPython-->

## <font color=red>處理步驟
------
![image.png](attachment:image.png)

## <font color=red>請先設立一個新的virtualenv

## <font color=red>然後安裝下列組件

In [ ]:
!pip install easyocr
!pip install imutils  #圖像路徑處理工具
!pip install matplotlib
!pip install opencv-python==4.5.4.60 --user

## <font color=red>導入工具

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr

## <font color=red>1. 把圖片讀進來，改變一下尺寸

In [ ]:
image = cv2.imread('JPG1.jpg')
image = imutils.resize(image, width=500)
cv2.imshow("Original Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <font color=red>2.轉灰階

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("1 - Grayscale Conversion", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <font color=red>4.在保持edge的狀態下，去除noise

In [ ]:
gray = cv2.bilateralFilter(gray, 11, 17, 17)
cv2.imshow("2 - Bilateral Filter", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <font color=red>5.用Canny取得邊界圖

In [ ]:
edged = cv2.Canny(gray, 170, 200)
cv2.imshow("3 - Canny Edges", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <font color=red>6.計算contour

In [ ]:
cnts, new  = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
img1 = image.copy()
cv2.drawContours(img1, cnts, -1, (0,255,0), 3)
cv2.imshow("4- All Contours", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <font color=red>7.利用面積大小，計算出面積最大前30名的contour

In [ ]:
cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30]

img2 = image.copy()
cv2.drawContours(img2, cnts, -1, (0,255,0), 3)
cv2.imshow("5- Top 30 Contours", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <font color=red>8.把車牌摘下來

In [ ]:
cropped_image= None
approx = None
for c in cnts:
        #開始對每個contour做邊界辨識
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        #挑選多邊形尋找結果是有4個corners的
        if len(approx) == 4:
            # Crop those contours and store it in Cropped Images folder
            x, y, w, h = cv2.boundingRect(c) #This will find out co-ord for plate
            cropped_image= gray[y:y + h, x:x + w] #Create new image
            break

cv2.imshow("Cropped Image ", cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(approx )

## <font color=red>9.開始認字

In [ ]:
reader = easyocr.Reader(['en'])
result = reader.readtext(cropped_image)
print(result)

### 如果easyocr運轉有問題，opencv要降轉
```
pip install opencv-python==4.5.4.60
```

## <font color=red>10.標示一下

```
若要在圖片上加上文字，可以使用 cv2.putText 函數：

cv2.putText(影像, 文字, 座標, 字型, 大小, 顏色, 線條寬度, 線條種類)
```

In [ ]:
#把辨別的文字取出來
text = result[0][-2]
#設定一下字體
font = cv2.FONT_HERSHEY_SIMPLEX
#把文字寫上圖片
#org是字的座標，參考approx的位置，往下移動60
res = cv2.putText(image, text=text, org=(approx[0][0][0], approx[1][0][1]+60), fontFace=font, fontScale=1, color=(0,255,0), thickness=2, lineType=cv2.LINE_AA)
#把車牌框做出來
res = cv2.rectangle(image, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3)
plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))